# Lab 2: Network Intrusion Detection

In [1]:
# Imports
from matplotlib import pyplot as plt

import pandas
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import StratifiedKFold

from sklearn.naive_bayes import GaussianNB

from sklearn.decomposition import RandomizedPCA 
from sklearn.decomposition import PCA
from sklearn.lda import LDA

from sklearn.pipeline import Pipeline
from sklearn import metrics as mt

import seaborn as sns
import pandas as pd
import numpy as np


%matplotlib inline

C:\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


## Data Preparation:

* Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

In [2]:
# Load UNSW_NB15 into a Pandas dataframe
df = pd.read_csv('UNSW_NB15_training_set.csv', encoding='utf-8-sig')

# Lets remove attributes that are not useful to us during this first analysis pass
non_useful_features_list = ['id', 'attack_cat']
# id: n internal variable to just ref an obseration. deemed not usefl
# attack_cat: first try and just predict the label. 
#             It will obviously 1:1 correlate with label
#             We can circle back and swap it out with label 
#             to see if we get any better accuracy on an 
#             on an attack type level
for feature in non_useful_features_list:
    if feature in df:
        df.drop(feature, axis=1, inplace=True)  # Lets drop id as it is an internal variable to just ref an obseratio
        
# Overwrite the existing dataframe with the new dataframe that does not contain the 
# four unwanted records and confirm we have 4 less records (shold have 82328 observations)
if "is_ftp_login" in df:
    df = df[df.is_ftp_login != 2]
    if len(df) == 82328:
        print ("duplicate record deleted successfully: " + str(len(df)) + " observations remaining" )
        
# Check to see if non useful features still exist in dataframe, if so, we did something wrong
for feature in non_useful_features_list:
    if feature in df:
        print ("[" + feature + "]" + "still found, check removal code. (Should not see this)" )

duplicate record deleted successfully: 82328 observations remaining


* Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

In [3]:
df.describe()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
count,82328.000000,82328.000000,82328.000000,8.232800e+04,8.232800e+04,8.232800e+04,82328.000000,82328.00000,8.232800e+04,8.232800e+04,...,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000,82328.000000
mean,1.006783,18.666893,17.546303,7.994267e+03,1.323440e+04,8.241489e+04,180.973448,95.70541,6.455215e+07,6.305771e+05,...,4.929040,3.663092,7.456528,0.008187,0.008284,0.129749,6.468480,9.164610,0.011126,0.550578
std,4.710557,133.919593,115.576881,1.716464e+05,1.514751e+05,1.486229e+05,101.512436,116.66547,1.798656e+08,2.393055e+06,...,8.389724,5.915518,11.415443,0.090110,0.091439,0.638697,8.544117,11.121571,0.104893,0.497438
min,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000e+00,0.000000,0.00000,0.000000e+00,0.000000e+00,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000008,2.000000,0.000000,1.140000e+02,0.000000e+00,2.860585e+01,62.000000,0.00000,1.120356e+04,0.000000e+00,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000
50%,0.014120,6.000000,2.000000,5.340000e+02,1.780000e+02,2.651198e+03,254.000000,29.00000,5.770751e+05,2.112632e+03,...,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,5.000000,0.000000,1.000000
75%,0.719362,12.000000,10.000000,1.280000e+03,9.560000e+02,1.111111e+05,254.000000,252.00000,6.514286e+07,1.585818e+04,...,4.000000,3.000000,6.000000,0.000000,0.000000,0.000000,7.000000,11.000000,0.000000,1.000000
max,59.999989,10646.000000,11018.000000,1.435577e+07,1.465753e+07,1.000000e+06,255.000000,253.00000,5.268000e+09,2.082111e+07,...,59.000000,38.000000,63.000000,1.000000,2.000000,16.000000,60.000000,62.000000,1.000000,1.000000


## Modeling and Evaluation:
* Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

*  Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

*  Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms! 

In [5]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

dataframe = pandas.read_csv(url, names=names)
array = dataframe.values

X = array[:,0:8]
Y = array[:,8]

num_folds = 10
num_instances = len(X)
seed = 7
num_trees = 100
max_features = 3

kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)

results = cross_validation.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.769480519481


In [7]:
array = df.values

X = array[:,0:41]
Y = array[:,41]

num_folds = 10
num_instances = len(X)

seed = 7
num_trees = 100
max_features = 3

kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)

results = cross_validation.cross_val_score(model, X, Y, cv=kfold)
#print( str(results.mean()) )



   




*  Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

* Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation. 

* Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

## Deployment:
* How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

## Exceptional Work:

In [37]:
df_copy = df.select_dtypes(['float64', 'int64'])

y = df.attack_cat


#######################################################
# Percentage of variance explained by first component
#######################################################
pca = PCA(n_components=1)
x_pca = pca.fit(df_copy).transform(df_copy)

# Percentage of variance explained for each component
print('explained variance ratio (first component): %3.2f'
      % (100 * pca.explained_variance_ratio_))


############################################################
# Percentage of variance explained for first two components
############################################################
pca = PCA(n_components=2)
x_pca = pca.fit(df_copy).transform(df_copy)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % (100 * pca.explained_variance_ratio_) )

 
##############################################################
# Percentage of variance explained for first three components
##############################################################
pca = PCA(n_components=3)
x_pca = pca.fit(df_copy).transform(df_copy) 

# Percentage of variance explained for each components
print('explained variance ratio (first three components): %s'
      % (100 * pca.explained_variance_ratio_) )



explained variance ratio (first component): 80.63
explained variance ratio (first two components): [ 80.62648708  18.61891863]
explained variance ratio (first three components): [ 80.62648708  18.61891863   0.75445077]
